# Spatial Features - Download SPFEAS results
Spatial features (spfeas) is a landcover classification tool focused on leveraging imagery textures for classifying landcover. It has been used in the World Bank for both quantifying informal housing and estimating poverty

Running spatial features is a 5 part process:

0. Setting up gbdx for spatial features
1. Finding imagery
2. Runnning spfeas
3. **Downloading results from AWS**
4. Stacking results
5. Running classification

### Downloaded data are HUGE - this should be done on one of the gaming machines, not your personal PC

In [1]:
outputFolder = "D:/Addis/spfeas"
s3Path = "haiti" #Set the folder for the output. Found at s3browser.geobigdata.io. ie - bps/Mauritania/Nouakchott
s3File = "C:/Temp/s3Contents.txt" #This is used for temporary execution
s3Exec = "C:/Temp/s3Contents.bat"
downloadFolder = "haiti" # Change to clippedRaster or maybe image to download the base imagery

In [2]:
import sys, os, inspect

from gbdxtools import Interface
from gbdxtools import CatalogImage
from shapely.geometry import box

cmd_folder = os.path.dirname(os.getcwd())
if cmd_folder not in sys.path:
    sys.path.insert(0, cmd_folder)

from GOST_GBDx_Tools import gbdxTasks
from GOST_GBDx_Tools import gbdxURL_misc

#In order for the interface to be properly authenticated, follow instructions here:
#   http://gbdxtools.readthedocs.io/en/latest/user_guide.html
#   For Ben, the .gbdx-config file belongs in C:\Users\WB411133 (CAUSE no one else qill f%*$&ing tell you that)
gbdx = Interface()
gbdx.s3.info
curTasks = gbdxTasks.GOSTTasks(gbdx)
gbdxUrl = gbdxURL_misc.gbdxURL(gbdx)


In [3]:
# Get list of ALL files in the defined AWS folder
downloadFolder = "haiti" 
xx = gbdxUrl.listS3Contents("s3://gbd-customer-data/%s/%s/" % (gbdxUrl.prefix, s3Path), outFile=s3File)
gbdxUrl.executeAWS_file(xx, s3Exec)
#It is possible to manually edit the s3File, if you want

In [7]:
#Download results from s3 file
with open(s3File) as inFile:
    for f in inFile:
        splitFolder = f.split(" ")
        imageName = splitFolder[-1].replace("\n", "")
        try:
            #Each line in this folder represents a processed image
            curFolder = "s3://gbd-customer-data/%s/%s/%s/%s" % (gbdxUrl.prefix, s3Path, imageName, downloadFolder)
            #Download spfeas Results
            curOut = os.path.join(outputFolder, imageName)
            try:
                os.mkdir(curOut)
                xx = gbdxUrl.downloadS3Contents(curFolder, curOut, recursive=True)
                gbdxUrl.executeAWS_file(xx,s3Exec)
            except:
                print("%s already exists" % curOut)
        except:
            print("Error processing %s" % f)

D:/Addis/spfeas\image/ already exists
